Financial Stock Price Dataset: Daily or intraday data for various stocks, which can be obtained from financial databases like Yahoo Finance or Alpha Vantage.

In [1]:
# Example: Fetching historical stock prices from Yahoo Finance
import yfinance as yf

# Define the stock ticker and time period
ticker = 'HPE'
data = yf.download(ticker, start='2023-01-01', end='2024-05-01')
data["Company"] = "Hewlett Packard Enterprise"

# ticker = 'MSFT'
# data2 = yf.download(ticker, start='2023-01-01', end='2024-05-01')
# data2["Company"] = "Microsoft"

print(data.head())

[*********************100%***********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close    Volume  \
Date                                                                          
2023-01-03  16.090000  16.139999  15.830000  16.059999  15.367215  18233900   
2023-01-04  16.150000  16.525000  16.120001  16.420000  15.711686  15865500   
2023-01-05  16.290001  16.495001  16.184999  16.450001  15.740394  13099500   
2023-01-06  16.680000  17.209999  16.670000  17.110001  16.371922  15242500   
2023-01-09  17.100000  17.250000  16.889999  16.959999  16.228392  19294700   

                               Company  
Date                                    
2023-01-03  Hewlett Packard Enterprise  
2023-01-04  Hewlett Packard Enterprise  
2023-01-05  Hewlett Packard Enterprise  
2023-01-06  Hewlett Packard Enterprise  
2023-01-09  Hewlett Packard Enterprise  


In [2]:
data.to_csv("data/HPE_stock.csv")

HPE Stocks data

In [4]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,333.000000,333.000000,333.000000,333.000000,333.000000,3.330000e+02
mean,16.220631,16.418802,16.045859,16.234535,15.830550,1.267898e+07
std,1.090092,1.123950,1.070711,1.085906,1.145174,6.267215e+06
min,13.700000,13.900000,13.655000,13.780000,13.300308,3.788800e+06
25%,15.450000,15.620000,15.260000,15.500000,15.077985,8.779800e+06
50%,16.309999,16.495001,16.180000,16.389999,15.826511,1.098290e+07
75%,17.030001,17.245001,16.875000,17.049999,16.751013,1.461540e+07
max,18.790001,20.070000,18.340000,18.770000,18.521847,5.125090e+07


For Stock Prices:

- Handle missing values by forward-filling or removing them.
- Normalize prices or returns to bring them into a common scale.

In [5]:
# Example: Cleaning and normalizing stock price data
data.fillna(method='ffill', inplace=True)  # Forward-fill missing values
data['Returns'] = data['Adj Close'].pct_change().fillna(0)  # Calculate daily returns
data['Normalized_Returns'] = (data['Returns'] - data['Returns'].mean()) / data['Returns'].std()

print(data[['Returns', 'Normalized_Returns']].head())


             Returns  Normalized_Returns
Date                                    
2023-01-03  0.000000           -0.024436
2023-01-04  0.022416            1.149628
2023-01-05  0.001827            0.071263
2023-01-06  0.040121            2.076976
2023-01-09 -0.008767           -0.483610


/tmp/ipykernel_358030/2474532667.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)  # Forward-fill missing values


Financial News Articles: News articles related to the stock market, which can be sourced from news aggregators like Google News, or specific financial news providers like Bloomberg or Reuters.

HPE news

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import traceback

def item_search(item, limit, page):
    stocknews = f"https://www.thestar.com.my/search/?q={item}&qsort=oldest&qrec={limit}&qstockcode=&pgno={page}"

    html = requests.get(stocknews).text

    soup = BeautifulSoup(html, 'html.parser')
    return soup

def get_details(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')

    content = soup.find('div', {'id':'story-body'})
    if content:
        content.get_text(strip=True)
        return content
    else:
        return ""
def star_new_crawler(page, search_query, limit):

    title = []
    links = []
    premium = []
    new_type = []
    contents = []
    publishedDate = []
    while True:
        print(page)
        try:
            result = item_search(search_query, limit, page)
            title += [x.get_text(strip=True) for x in result.find_all("h2", {"class": "f18"})]
            links += [x.find('a', {"data-content-type": "Article"})['href'] for x in
                     result.find_all("h2", {"class": "f18"})]
            premium += [x.get_text(strip=True) for x in result.find_all("span", {"class": "biz-icon"})]
            new_type += [x.get_text(strip=True) for x in result.find_all("a", {"class": "kicker"})]
            contents += [get_details(x) for x in links]
            publishedDate += [x.get_text(strip=True) for x in result.find_all("span", {"class": "timestamp"})]
         
            if len(title) == 0:
                break
        except Exception as e:
            print(e)
            traceback.print_exc()
    
        page += 1

    return pd.DataFrame({'new_type': new_type, 'title': title, 'premium': premium, 'links': links, 'published_data': publishedDate,
                 'contents': contents}).to_excel(f'{search_query}.xlsx', index=False)

if __name__ == '__main__':
    page = 1
    search_query = 'Hewlett Packard Enterprise'
    limit = 30
    df = star_new_crawler(page, search_query, limit)

1


In [24]:
import newspaper
from newspaper import Article
import pandas as pd
from datetime import datetime

# Define news sources to scrape
news_sources = news_sources = [
    "https://www.reuters.com/technology/",
    "https://www.cnbc.com/technology/",
    "https://www.bloomberg.com/technology"
    "https://www.forbes.com/sites/technology/"
    "https://www.businesstoday.in/latest/economy/"
]
# Keywords to filter articles
keywords = ["HPE", "Hewlett Packard Enterprise"]#,"Moonshot","GreenLake","HPE for HPC","Aruba","HPE Ezmeral","HPE Pointnext","HPE Financial Services"]

# Date range for filtering
start_date = datetime(2021, 8, 1)
end_date = datetime(2024, 9, 1)

# Function to collect articles
def collect_articles(news_sources, keywords, start_date, end_date):
    articles = []
    
    for source in news_sources:
        paper = newspaper.build(source, memoize_articles=False)
        
        for article in paper.articles:
            try:
                article.download()
                article.parse()

                # Check if the article's publication date is within the desired range
                if article.publish_date and start_date <= article.publish_date <= end_date:
                    # Check if the article contains any of the keywords
                    if any(keyword in article.text for keyword in keywords):
                        print(article.title)
                        articles.append({
                            "title": article.title,
                            "date": article.publish_date,
                            "text": article.text,
                            "source": source
                        })
            except Exception as e:
                print(f"Failed to download article: {e}")
    
    return articles

# Collect articles
articles = collect_articles(news_sources, keywords, start_date, end_date)

# Convert to DataFrame for easier handling
df = pd.DataFrame(articles)
df.dropna(subset=['date'], inplace=True)  # Drop articles without a publish date

print(df.head())

# Save the dataset to a CSV file
#df.to_csv('apple_microsoft_financial_news.csv', index=False)


Failed to download article: can't compare offset-naive and offset-aware datetimes
Failed to download article: can't compare offset-naive and offset-aware datetimes
Failed to download article: can't compare offset-naive and offset-aware datetimes
Failed to download article: can't compare offset-naive and offset-aware datetimes
Failed to download article: can't compare offset-naive and offset-aware datetimes
Failed to download article: can't compare offset-naive and offset-aware datetimes
Failed to download article: can't compare offset-naive and offset-aware datetimes
Failed to download article: can't compare offset-naive and offset-aware datetimes
Failed to download article: can't compare offset-naive and offset-aware datetimes
Failed to download article: can't compare offset-naive and offset-aware datetimes
Failed to download article: can't compare offset-naive and offset-aware datetimes
Failed to download article: can't compare offset-naive and offset-aware datetimes
Failed to downlo

In [25]:
df.head()

,title,date,text,source
0,"The stock market should be great, not on a kni...",2024-08-11,"When I look at Thursday's rally, I am beginnin...",https://www.cnbc.com/technology/


In [26]:
df.to_csv('data/hpe_financial_news.csv', index=False)